In [91]:
import sys
import os
root = os.path.abspath(os.path.join(".."))
sys.path.append(root)

from src.data.preprocessing_and_features import (
    load_data,
    impute_position_change_type,
    fill_percent_fields,
    populate_missing_fields_from_stocks,
    fill_missing_from_holdings_with_two_matches,
    fill_avg_price,
    fill_current_shares,
    engineer_holdings_features,
    engineer_org_features,

)

In [92]:
import pandas as pd

orgs_csv_path = "../data/raw/orgs.csv"

holdings_csv_path = "../data/raw/holdings.csv"



df_orgs, df_holdings = load_data(orgs_csv_path, holdings_csv_path)
# Display the first few rows of each DataFrame

print("Orgs Data:")

display(df_orgs.head())



print("Holdings Data:")
display(df_holdings.head())

Orgs Data:


,email_extension,org_id,cik,ticker,stock_id,filer_id,bio,org_type
0,solelpartners.com,75b97ec2-ddb1-4a7a-b97a-359d9658e97d,1790604.0,NaN,NaN,463912.0,Solel Partners is a hedge fund manager based i...,INSTITUTIONAL_INVESTOR
1,alvariuminvestments.com,3b92ee8f-9761-484b-ae78-2f3d30eb19dd,NaN,ALTI,6941.0,NaN,"AlTi Global, Inc. is a publicly traded interna...",PUBLIC_COMPANY
2,aprio.com,16315875-6b1e-4adf-90c1-c6f16892bc16,1803519.0,NaN,NaN,463917.0,Aprio is a premier business advisory and accou...,INSTITUTIONAL_INVESTOR
3,ctcmycfo.com,259600df-96b9-4117-ad22-2fa07dea2528,1673962.0,NaN,NaN,336130.0,BMO Family Office is an integrated wealth mana...,INSTITUTIONAL_INVESTOR
4,boeing.com,c2696c82-1847-4cef-9665-6f893006579e,NaN,BA,210.0,NaN,Boeing Company (The) is a leading global aeros...,PUBLIC_COMPANY


Holdings Data:


,email_extension,id,org_id,filer_id,filer_name,stock_id,stock_name,stock_ticker,security_type,shares_change,...,industry,percent_ownership,filer_street_address,filer_city,filer_state,filer_zip_code,avg_price,percent_change,quarter_id_owned,quarter_end_price
0,solelpartners.com,b5e6446d-a8ec-470d-b234-e677f28e3d4c,75b97ec2-ddb1-4a7a-b97a-359d9658e97d,463912.0,SOLEL PARTNERS LP,221.0,CVS Health Corp,CVS,SH,625500.0,...,RETAIL - FOOD & DRUG,0.0897,NaN,NaN,NaN,NaN,62.0177,124.26,NaN,NaN
1,paradigm.com,bf8029ca-24b2-48c0-b064-6d639fa32bf3,a60fe97c-3839-463d-8937-68e8e0cee430,2158.0,PARADIGM CAPITAL MANAGEMENT IN,6948.0,American Woodmark Corp,AMWD,SH,301189.0,...,CONSTRUCTION & ENGINEERING,2.0007,NaN,NaN,NaN,NaN,90.8964,NaN,NaN,NaN
2,ruffer.co.uk,15d99797-69cc-4bf8-873d-04ae9a470d61,a9897469-0b3b-4de3-94b2-fc3d84ef665f,2508.0,RUFFER LLP,6021.0,First Solar Inc,FSLR,SH,30480.0,...,ELECTRICAL EQUIPMENT,0.0285,NaN,NaN,NaN,NaN,207.1650,NaN,NaN,NaN
3,solelpartners.com,a861cb00-eeab-40a0-82ca-3959271e5a2f,75b97ec2-ddb1-4a7a-b97a-359d9658e97d,463912.0,SOLEL PARTNERS LP,211769.0,Grocery Outlet Holding Corp,GO,SH,813627.0,...,FOOD PRODUCTS,2.4902,NaN,NaN,NaN,NaN,22.8637,50.02,NaN,NaN
4,solelpartners.com,145f8c19-c53b-481e-8941-face43a8d48c,75b97ec2-ddb1-4a7a-b97a-359d9658e97d,463912.0,SOLEL PARTNERS LP,11343.0,Cigna Holding Co,CI,SH,-2900.0,...,INSURANCE,0.0387,NaN,NaN,NaN,NaN,226.5949,-2.61,NaN,NaN


In [93]:
df_holdings.isna().all()

email_extension                  False
id                               False
org_id                           False
filer_id                         False
filer_name                       False
stock_id                         False
stock_name                       False
stock_ticker                     False
security_type                    False
shares_change                    False
position_change_type             False
current_ranking                  False
previous_ranking                 False
current_percent_of_portfolio     False
previous_percent_of_portfolio    False
current_mv                       False
previous_mv                      False
current_shares                   False
previous_shares                  False
source_date                      False
source                            True
sector                           False
industry                         False
percent_ownership                False
filer_street_address              True
filer_city               

In [94]:
df_orgs.isna().all()

email_extension    False
org_id             False
cik                False
ticker             False
stock_id           False
filer_id           False
bio                False
org_type           False
dtype: bool

In [95]:
df_holdings[['stock_ticker']]

,stock_ticker
0,CVS
1,AMWD
2,FSLR
3,GO
4,CI
...,...
174373,NaN
174374,NaN
174375,NaN
174376,NaN


In [96]:
pd.DataFrame(df_holdings[['stock_ticker']].value_counts())

,count
stock_ticker,
MSFT,713
AMZN,671
META,591
AAPL,584
NVDA,583
...,...
0R87N:MEXI,1
ZURAW,1
ZSANQ,1


In [97]:
print(f"orgs rows {len(df_orgs)} and holdings rows {len(df_holdings)}")

orgs rows 3630 and holdings rows 174378


In [98]:
df_holdings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 174378 entries, 0 to 174377
Data columns (total 32 columns):
 #   Column                         Non-Null Count   Dtype         
---  ------                         --------------   -----         
 0   email_extension                174378 non-null  object        
 1   id                             173949 non-null  object        
 2   org_id                         173949 non-null  object        
 3   filer_id                       173949 non-null  float64       
 4   filer_name                     172823 non-null  object        
 5   stock_id                       173949 non-null  float64       
 6   stock_name                     173949 non-null  object        
 7   stock_ticker                   173927 non-null  object        
 8   security_type                  173949 non-null  object        
 9   shares_change                  173949 non-null  float64       
 10  position_change_type           153606 non-null  object        
 11  

In [99]:
df_holdings = df_holdings.drop(
    columns=[
        "source",
        "quarter_id_owned",
        "quarter_end_price",
        "filer_street_address",
        "filer_city",
        "filer_state",
        "filer_zip_code",
    ]
)  # no data or deeming unimportant
df_holdings = populate_missing_fields_from_stocks(df_holdings, "../data/raw/stocks.csv")
df_holdings = df_holdings.dropna(
    subset=["stock_ticker"]
)  # perhaps drop or set to 0/-1 as per stocks.csv? only 20ish rows so drop but need to deepdive in the future
df_holdings[
    [
        "current_mv",
        "previous_mv",
        "previous_percent_of_portfolio",
        "current_percent_of_portfolio",
    ]
] = df_holdings[
    [
        "current_mv",
        "previous_mv",
        "previous_percent_of_portfolio",
        "current_percent_of_portfolio",
    ]
].fillna(
    0.0
)
df_holdings["current_ranking"] = df_holdings["current_ranking"].fillna(999999)
df_holdings["previous_ranking"] = df_holdings["previous_ranking"].fillna(999999)
df_holdings["filer_id"] = df_holdings["filer_id"].fillna(0)
df_holdings["filer_name"] = df_holdings["filer_name"].fillna("unassigned filer")
df_holdings = df_holdings[
    ~df_holdings["stock_ticker"].isin(
        ["0", "660", "0R87N:MEXI", "11G.OLD:FWB2", "9688.HK"]
    )
]
df_holdings["previous_shares"] = df_holdings["previous_shares"].fillna(0)
df_holdings["current_shares"] = df_holdings.apply(fill_current_shares, axis=1)
# Now drop rows where current_shares is still NaN
df_holdings = df_holdings.dropna(subset=["current_shares"])
df_holdings["position_change_type"] = df_holdings.apply(
    impute_position_change_type, axis=1
)
df_holdings[["sector", "industry"]] = df_holdings[["sector", "industry"]].fillna(
    "Unknown"
)
df_holdings = df_holdings.apply(fill_percent_fields, axis=1)
sector_medians = df_holdings.groupby("sector")["avg_price"].median()
df_holdings["avg_price"] = df_holdings.apply(
    lambda row: fill_avg_price(row, sector_medians), axis=1
)
df_holdings.info()

<class 'pandas.core.frame.DataFrame'>
Index: 173922 entries, 0 to 173948
Data columns (total 25 columns):
 #   Column                         Non-Null Count   Dtype         
---  ------                         --------------   -----         
 0   email_extension                173922 non-null  object        
 1   id                             173922 non-null  object        
 2   org_id                         173922 non-null  object        
 3   filer_id                       173922 non-null  float64       
 4   filer_name                     173922 non-null  object        
 5   stock_id                       173922 non-null  float64       
 6   stock_name                     173922 non-null  object        
 7   stock_ticker                   173922 non-null  object        
 8   security_type                  173922 non-null  object        
 9   shares_change                  173922 non-null  float64       
 10  position_change_type           173922 non-null  object        
 11  curre

In [100]:
df_orgs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3630 entries, 0 to 3629
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   email_extension  3630 non-null   object 
 1   org_id           3630 non-null   object 
 2   cik              1691 non-null   float64
 3   ticker           1936 non-null   object 
 4   stock_id         1936 non-null   float64
 5   filer_id         1691 non-null   float64
 6   bio              3630 non-null   object 
 7   org_type         3630 non-null   object 
dtypes: float64(3), object(5)
memory usage: 227.0+ KB


In [101]:
df_orgs = df_orgs.rename(columns={"ticker": "stock_ticker"})
df_orgs = df_orgs.drop(columns=["cik"])
df_orgs = fill_missing_from_holdings_with_two_matches(
    df_orgs, df_holdings
)  # big lift for only filer_id as we have more stock_id-ticker pairs; here you could use LLM to decipher missing pieces instead
df_orgs["stock_ticker"] = df_orgs["stock_ticker"].fillna(
    "Unknown"
)  # about 1/3 unknown!!
df_orgs["stock_id"] = df_orgs["stock_id"].fillna(0)  # about 1/3 unknown!!
df_orgs = df_orgs.dropna(subset=["filer_id"])
df_orgs.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3386 entries, 0 to 3629
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   email_extension  3386 non-null   object 
 1   org_id           3386 non-null   object 
 2   stock_ticker     3386 non-null   object 
 3   stock_id         3386 non-null   float64
 4   filer_id         3386 non-null   float64
 5   bio              3386 non-null   object 
 6   org_type         3386 non-null   object 
dtypes: float64(2), object(5)
memory usage: 211.6+ KB


In [102]:
df_holdings = engineer_holdings_features(df_holdings)
df_orgs = engineer_org_features(df_orgs, df_holdings)

In [103]:
df_orgs.head()

,email_extension,org_id,stock_ticker,stock_id,filer_id,bio,org_type,num_holdings,total_mv,avg_investment_size,most_invested_sector,diversification_score
0,solelpartners.com,75b97ec2-ddb1-4a7a-b97a-359d9658e97d,Unknown,0.0,463912.0,Solel Partners is a hedge fund manager based i...,INSTITUTIONAL_INVESTOR,21.0,4.908855e+08,2.337550e+07,FINANCE,6.0
1,alvariuminvestments.com,3b92ee8f-9761-484b-ae78-2f3d30eb19dd,ALTI,6941.0,1431.0,"AlTi Global, Inc. is a publicly traded interna...",PUBLIC_COMPANY,1.0,0.000000e+00,0.000000e+00,MATERIALS,1.0
2,aprio.com,16315875-6b1e-4adf-90c1-c6f16892bc16,Unknown,0.0,463917.0,Aprio is a premier business advisory and accou...,INSTITUTIONAL_INVESTOR,100.0,6.103336e+08,6.103336e+06,FINANCE,11.0
3,ctcmycfo.com,259600df-96b9-4117-ad22-2fa07dea2528,Unknown,0.0,336130.0,BMO Family Office is an integrated wealth mana...,INSTITUTIONAL_INVESTOR,100.0,2.822213e+09,2.822213e+07,COMMUNICATIONS,11.0
4,boeing.com,c2696c82-1847-4cef-9665-6f893006579e,BA,210.0,35.0,Boeing Company (The) is a leading global aeros...,PUBLIC_COMPANY,1.0,6.184527e+10,6.184527e+08,INDUSTRIALS,1.0


In [104]:
df_orgs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3386 entries, 0 to 3385
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   email_extension        3386 non-null   object 
 1   org_id                 3386 non-null   object 
 2   stock_ticker           3386 non-null   object 
 3   stock_id               3386 non-null   float64
 4   filer_id               3386 non-null   float64
 5   bio                    3386 non-null   object 
 6   org_type               3386 non-null   object 
 7   num_holdings           3386 non-null   float64
 8   total_mv               3386 non-null   float64
 9   avg_investment_size    3386 non-null   float64
 10  most_invested_sector   3386 non-null   object 
 11  diversification_score  3386 non-null   float64
dtypes: float64(6), object(6)
memory usage: 317.6+ KB


In [105]:
df_holdings.info()

<class 'pandas.core.frame.DataFrame'>
Index: 173922 entries, 0 to 173948
Data columns (total 29 columns):
 #   Column                         Non-Null Count   Dtype         
---  ------                         --------------   -----         
 0   email_extension                173922 non-null  object        
 1   id                             173922 non-null  object        
 2   org_id                         173922 non-null  object        
 3   filer_id                       173922 non-null  float64       
 4   filer_name                     173922 non-null  object        
 5   stock_id                       173922 non-null  float64       
 6   stock_name                     173922 non-null  object        
 7   stock_ticker                   173922 non-null  object        
 8   security_type                  173922 non-null  object        
 9   shares_change                  173922 non-null  float64       
 10  position_change_type           173922 non-null  object        
 11  curre

In [106]:
import pandas as pd

orgs_csv_path = "../data/processed/orgs_processed.csv"

holdings_csv_path = "../data/processed/holdings_processed.csv"



df_orgs_good, df_holds_good = load_data(orgs_csv_path, holdings_csv_path)
# Display the first few rows of each DataFrame

In [107]:
df_holds_good[df_holds_good['stock_ticker'].isna()]

,org_id_encoded,email_extension_encoded,stock_ticker,filer_id,stock_id,shares_change,position_change_type,current_ranking,previous_ranking,current_percent_of_portfolio,...,industry_TRUCKING,industry_TRUSTS & THRIFTS,industry_UTILITIES - ELECTRIC,industry_UTILITIES - GAS,industry_UTILITIES - MULTILINE,industry_UTILITIES - WATER,industry_Unknown,industry_WOOD & PAPER,industry_WORLD ALLOCATION,industry_WORLD STOCK
27068,322,2147,NaN,2111.0,-1.0,0.258532,1,17,1,-0.176200,...,False,False,False,False,False,False,False,False,False,False
31392,2445,530,NaN,39281.0,-1.0,0.108824,1,17,1,-0.176200,...,False,False,False,False,False,False,False,False,False,False
37283,295,562,NaN,502.0,-1.0,0.497495,1,17,1,-0.176200,...,False,False,False,False,False,False,False,False,False,False
54592,2404,499,NaN,276.0,-1.0,-0.012138,1,17,1,-0.176200,...,False,False,False,False,False,False,False,False,False,False
58989,2502,2040,NaN,2019.0,-1.0,2.101038,1,17,1,-0.176200,...,False,False,False,False,False,False,False,False,False,False
70773,2621,1963,NaN,463610.0,292646.0,-0.001062,0,0,1,13.753893,...,False,False,False,False,False,False,False,False,False,False
70774,2621,1963,NaN,0.0,292646.0,0.144308,1,0,1,13.115351,...,False,False,False,False,False,False,False,False,False,False
70775,2621,1963,NaN,463608.0,292646.0,0.079081,0,0,1,13.753893,...,False,False,False,False,False,False,False,False,False,False
70776,2621,1963,NaN,623984.0,292646.0,-0.052252,2,0,1,13.753893,...,False,False,False,False,False,False,False,False,False,False
70777,2621,1963,NaN,3010.0,292646.0,-0.012142,0,5027,5111,-0.176200,...,False,False,False,False,False,False,False,False,False,False


In [108]:
df_orgs.isna().all()

email_extension          False
org_id                   False
stock_ticker             False
stock_id                 False
filer_id                 False
bio                      False
org_type                 False
num_holdings             False
total_mv                 False
avg_investment_size      False
most_invested_sector     False
diversification_score    False
dtype: bool

In [109]:
df_holdings.isna().all()

email_extension                  False
id                               False
org_id                           False
filer_id                         False
filer_name                       False
stock_id                         False
stock_name                       False
stock_ticker                     False
security_type                    False
shares_change                    False
position_change_type             False
current_ranking                  False
previous_ranking                 False
current_percent_of_portfolio     False
previous_percent_of_portfolio    False
current_mv                       False
previous_mv                      False
current_shares                   False
previous_shares                  False
source_date                      False
sector                           False
industry                         False
percent_ownership                False
avg_price                        False
percent_change                   False
growth_of_mv             

In [110]:
df_orgs.head()

,email_extension,org_id,stock_ticker,stock_id,filer_id,bio,org_type,num_holdings,total_mv,avg_investment_size,most_invested_sector,diversification_score
0,solelpartners.com,75b97ec2-ddb1-4a7a-b97a-359d9658e97d,Unknown,0.0,463912.0,Solel Partners is a hedge fund manager based i...,INSTITUTIONAL_INVESTOR,21.0,4.908855e+08,2.337550e+07,FINANCE,6.0
1,alvariuminvestments.com,3b92ee8f-9761-484b-ae78-2f3d30eb19dd,ALTI,6941.0,1431.0,"AlTi Global, Inc. is a publicly traded interna...",PUBLIC_COMPANY,1.0,0.000000e+00,0.000000e+00,MATERIALS,1.0
2,aprio.com,16315875-6b1e-4adf-90c1-c6f16892bc16,Unknown,0.0,463917.0,Aprio is a premier business advisory and accou...,INSTITUTIONAL_INVESTOR,100.0,6.103336e+08,6.103336e+06,FINANCE,11.0
3,ctcmycfo.com,259600df-96b9-4117-ad22-2fa07dea2528,Unknown,0.0,336130.0,BMO Family Office is an integrated wealth mana...,INSTITUTIONAL_INVESTOR,100.0,2.822213e+09,2.822213e+07,COMMUNICATIONS,11.0
4,boeing.com,c2696c82-1847-4cef-9665-6f893006579e,BA,210.0,35.0,Boeing Company (The) is a leading global aeros...,PUBLIC_COMPANY,1.0,6.184527e+10,6.184527e+08,INDUSTRIALS,1.0
